In [1]:
import torch
from transformers import BertJapaneseTokenizer, BertModel

# 4-4 Transformers

- トークナイザでトークン化して、符号化（id化）して、BERTに入力できる形にする
- BERTに入力し、出力を得る

In [2]:
model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

In [3]:
tokenizer.tokenize("明日は自然言語処理の勉強をしよう。")

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。']

In [4]:
# 「##」はWordPieceによる分割の単語の最初以外のトークン
tokenizer.tokenize("明日はマシンラーニングの勉強をしよう。")

['明日', 'は', 'マシン', '##ラー', '##ニング', 'の', '勉強', 'を', 'しよ', 'う', '。']

In [5]:
tokenizer.tokenize("機械学習を中国語にすると机器学习だ。")

['機械', '学習', 'を', '中国', '語', 'に', 'する', 'と', '机', '器', '学', '[UNK]', 'だ', '。']

In [7]:
input_ids = tokenizer.encode("明日は自然言語処理の勉強をしよう。")
print(input_ids)

[2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 205, 8, 3]


In [8]:
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。', '[SEP]']

- 複数の文章をまとめて処理する際に、系列長を揃えるため`[PAD]`を足す
- `[PAD]`は本来の処理と関係ない余計なものなので、どの部分にAttentionをかけるかをあらわす attention_mask も用意する

In [9]:
text = "明日の天気は晴れだ。"
encoding = tokenizer(text, max_length=12, padding="max_length", truncation=True)
print("# encoding: ")
print(encoding)

# encoding: 
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}


In [10]:
tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
print(tokens)

['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


In [12]:
from pprint import pprint

text_list = ["明日の天気は晴れだ。", "パソコンが急に動かなくなった。"]
tmp_encoded = tokenizer(
    text_list,
    max_length=10,
    padding="max_length",
    truncation=True,
)

pprint(tmp_encoded)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0],
               [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 3]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}


In [13]:
tmp_encoded = tokenizer(
    text_list,
    padding="longest",
)

pprint(tmp_encoded)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0],
               [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 8, 3]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}


In [15]:
pprint(
    tokenizer(
        text_list,
        max_length=10,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[    2, 11475,     5, 11385,     9, 16577,    75,     8,     3,     0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     3]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


- BertModel
  - トークンに対するベクトルを得るクラス
  - 直接使うことはほぼない
  - 入出力関係は個別の言語タスク用モデルと似ているので、解説のため 

In [16]:
model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
bert = BertModel.from_pretrained(model_name)
bert = bert.cuda()

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [17]:
pprint(bert.config)

BertConfig {
  "_name_or_path": "tohoku-nlp/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.40.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



In [20]:
### BertModel に符号化(id化)した文章を入力することで、BERTの最終層の出力を得る ###
text_list = [
    "明日は自然言語処理の勉強をしよう。",
    "明日はマシーンラーニングの勉強をしよう。"
]

# 符号化
encoding = tokenizer(
    text_list,
    max_length=32,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)

# データをGPUに乗せる
encoding = {k: v.cuda() for k, v in encoding.items()}
# BERTでの処理
output = bert(**encoding)
# 最終層の出力
last_hidden_state = output.last_hidden_state

In [21]:
print(last_hidden_state.size())

torch.Size([2, 32, 768])


- バッチサイズ（文章の数）が2,
- 系列長が32,
- 隠れ状態（分散表現）が768